# Starting off with ipyvolume for 3D things

In [1]:
# preferable
#!conda install -c conda-forge ipyvolume

# you can also pip-install
#!pip install ipyvolume
# then you might need:
#!jupyter nbextension enable --py --sys-prefix ipyvolume
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [2]:
import ipyvolume

In [3]:
# quick scatter plot
import numpy as np
x, y, z = np.random.random((3,1000))
ipyvolume.quickscatter(x,y,z,size=1,marker='arrow')

In [4]:
# import our data that we saved from our simulations
from hermite_library import read_hermite_solution_from_file

In [19]:
# read in my saved simulation data
t_h, E_h, r_h, v_h = read_hermite_solution_from_file('myPlanetSystem_kepler101_solution1.txt')
#t_h, E_h, r_h, \
#v_h = read_hermite_solution_from_file('../lesson05/myPlanetSystem_kepler11_solution1.txt')

In [20]:
# reformat a little bit for plotting
x = r_h[:,0,:].ravel() # x-coordinate, all particles, flattened
y = r_h[:,1,:].ravel() # y-coor, all particles, flattened
z = r_h[:,2,:].ravel() # z-coor, all particles, flattened
x.shape, y.shape,z.shape

((15000,), (15000,), (15000,))

In [21]:
# quick scatter plot with my data
ipyvolume.quickscatter(x,y,z, size=1, marker='sphere')

In [22]:
# changing color of particles
ipyvolume.figure()
#colors = ['red','blue','green'] # colors for 3 particles
colors = ['red','blue','green','black','magenta','yellow','pink'] # colors for 3 particles
for i in range(r_h.shape[0]): # looping over all the particles in my file
    ipyvolume.scatter(r_h[i,0,:], #ith particle, x coord, all timesteps
                     r_h[i,1,:], # ith particle, y coord, all times
                     r_h[i,2,:], # ith particle, z coord, all times
                      color=colors[i], # pull from our color list 
                      marker='sphere',
                     size=0.1)
ipyvolume.show() # show the figure

In [23]:
# subsample
step = 1000 # plot every 1000th step
N = r_h.shape[2] # total number of steps

# let's plot velocity vectors
ipyvolume.figure()
#colors = ['red', 'blue', 'green']
colors = ['red','blue','green','black','magenta','yellow','pink'] # colors for 3 particles
for i in range(r_h.shape[0]): # loop over all particles
    ipyvolume.quiver(r_h[i,0,0:N:step], # ith particle, x coord, now plot every "step"
                    r_h[i,1,0:N:step], # ith particle, y
                    r_h[i,2,0:N:step], # ith particle, z
                    v_h[i,0,0:N:step], #   ith part, vx
                    v_h[i,1,0:N:step], # ith part, vy
                    v_h[i,2,0:N:step], # ith, part, vz
                    color=colors[i], size=2)
ipyvolume.show()

In [24]:
r_h[:,0,:].shape # x-coord

(3, 5000)

In [25]:
# take transpose
r_h[:,0,:].T.shape # a formating type thing, ipyvolume needs this shape for animations

(5000, 3)

In [26]:
nstep = 10 # do an animation every 10th timestep
N = r_h.shape[2] # number of total time steps

# subsample for less # of frames
#r = r_h[:,:,0:-1:nstep] # all particles, all coordinates, sub-sampled times
r = r_h[:,:,0:N:nstep] # all particles, all coordinates, sub-sampled times

r.shape

(3, 3, 500)

In [27]:
#           red        green    blue
colors = [ (1, 0, 0), (0,1,0), (0,0,1), (0,1,1), (0,0.5,1), (1,1,0),(0.5,1,1)]

In [28]:
len(colors)

7

In [29]:
# little function to format colors
from flip_colors import flip_colors

colors = flip_colors(colors,r) # taking my 3 

#outcolors = []
#for i in range(r.shape[2]):
#    outcolors.append(colors)
    
#colors = np.array(colors)

# order should be (times, points, colors)
#colors = np.transpose(colors, (0, 2, 1)) # flip the last axes


colors.shape

(500, 7, 3)

In [30]:
# finally make animation
ipyvolume.figure()

#                        x            y           z
s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                     marker='sphere',
                     color=colors) # remapped color array
ani = ipyvolume.animation_control(s, interval=200)
ipyvolume.show()

### Exercises
1. plot an animation of your system
1. Plot a quiver animation of your system

Bonus: how would you save this animation?

# widgets + ipyvolume

In [33]:
import ipywidgets

In [8]:
ipyvolume.figure()
s = ipyvolume.scatter(x,y,z, marker='sphere')
ipyvolume.show()

In [10]:
# make a widget to change the size of my spheres
size = ipywidgets.FloatSlider(min=0, max=30, step=0.1) # slider widget to control size
color = ipywidgets.ColorPicker() # color picker widget to control color

In [11]:
# link properties in my ipyvolume scatter plot with my widgets
ipywidgets.jslink((s,'size'), (size, 'value')) # scatter plot property, widget value
ipywidgets.jslink((s,'color'), (color, 'value'))

Link(source=(Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', line_material=ShaderMaterial(),…

In [12]:
# in a vertical stack take last ipyvolume figure, then size widget, then color widget
ipywidgets.VBox([ipyvolume.gcc(), size, color])

# Embed to html

In [13]:
# instead of displaying, save to an object
myOutFig = ipywidgets.VBox([ipyvolume.gcc(), size, color])

In [14]:
# the below is formatting to make sure our html page looks oke
ipyvolume.embed.layout = myOutFig.children[1].layout
ipyvolume.embed.embed_html.min_width='400px'

In [15]:
# finally we'll output to an html page
ipyvolume.embed.embed_html('myPage1.html',myOutFig, offline=False, devmode=False)

### Exercise
1. Redo with your simulation data

Other ideas:
1. What other widgets might you use?
1. How would you deploy this to the web? (Optional web-dev stuff)
1. How would you make an embeddable animation + widgets?

## Animation to upload to web

In [34]:
step = 10 # only plot every 10th timestep
N = r_h.shape[2] # all timesteps
r = r_h[:,:,0:N:step] # all particles, all coords, every stepth time step

ipyvolume.figure() # figure object

# create size widget
size = ipywidgets.FloatSlider(min=0,max=30,step=0.1, description='Size')

# scatter plot -> to make animation
s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T,
                     marker='sphere',
                     color=colors) # colors in (r,g,b) pairs

# link my scatter plot with my widget
ipywidgets.jslink((s,'size'),(size,'value')) # linking scatter plot size with size widget value

# make life more complicated -> animate!
ani = ipyvolume.animation_control(s, interval=200)

# finally, place with widgets Vbox
ipywidgets.VBox([ipyvolume.gcc(),size]) # last figure object generated, size widget


### REdo this with your simulation data
1. redo

If you want:
1. make an html embeddable object
1. deploy to the web using github